# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    print(root)
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

#print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
print(file_path_list)
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 


# for every filepath in the file path list 
for f in file_path_list:
    print(f)

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [3]:
# TO-DO: Create a Keyspace 
try:
    session.execute('''
    CREATE KEYSPACE IF NOT EXISTS events
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    ''')
except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [15]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_table_q1 = '''CREATE TABLE IF NOT EXISTS music_app_history 
(session_id int, item_in_session int, artist text, song_title text, song_length double,
PRIMARY KEY(session_id, item_in_session))'''

try:
    session.execute(create_table_q1)
except Exception as e:
    print(e)              

In [16]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = '''INSERT INTO music_app_history (session_id, item_in_session, artist, song_title, song_length)'''
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Task1: Getting the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4. 

##### Schema:
###### session_id, item_in_session, artist, song_title, song_length
###### PRIMARY KEY: session_id, item_in_session
###### I selected these schema because we need to filter with the sessionId and the item_in_session, so with this schema I will improve the queries responses with Apache Cassandra

In [17]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query_1 = "SELECT artist, song_title, song_length FROM music_app_history WHERE session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, '-', row.song_title, '-', row.song_length)

Faithless - Music Matters (Mark Knight Dub) - 495.3073


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [18]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_table_q2 = '''CREATE TABLE IF NOT EXISTS user_app_history 
(user_id int,  session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text,  
PRIMARY KEY((user_id, session_id), item_in_session))'''

try:
    session.execute(create_table_q2)
except Exception as e:
    print(e)                                 

In [19]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = '''INSERT INTO user_app_history (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)'''
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Task2: Getting the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### Schema:
###### user_id, session_id, item_in_session, artist, song_title, first_name, last_name
###### PRIMARY KEY: (user_id, session_id)
###### CLUSTERING COLUMN: item_in_session
###### I selected these schema because we need to filter with the user_id and the session_id, so they are the primary keys, and we have to sort by item_in_session, so I decided to select it as the clustering column, to improve the ordering and the queries responses

In [20]:
query_2 = "SELECT artist, song_title, first_name, last_name FROM user_app_history WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, '-', row.song_title, '-', row.first_name, row.last_name)

Down To The Bone - Keep On Keepin' On - Sylvie Cruz
Three Drives - Greece 2000 - Sylvie Cruz
Sebastien Tellier - Kilometer - Sylvie Cruz
Lonnie Gordon - Catch You Baby (Steve Pitron & Max Sanna Radio Edit) - Sylvie Cruz


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [21]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

create_table_q3 = '''CREATE TABLE IF NOT EXISTS specific_song_listen 
(user_id int, song_title text, first_name text, last_name text, PRIMARY KEY(user_id, song_title))'''

try:
    session.execute(create_table_q3)
except Exception as e:
    print(e)              

In [24]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = '''INSERT INTO specific_song_listen (user_id, song_title, first_name, last_name)'''
        query = query + " VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))

#### Task3: Getting the user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

##### Schema:
###### user_id, song_title, first_name, last_name
###### PRIMARY KEY: (user_id, song_title)
###### I selected these schema because we need to filter with the song_title, and the user_id to avoid information loss caused of the Apache Cassandra rules for partitioning.

In [28]:
query_3 = "SELECT first_name, last_name FROM specific_song_listen WHERE song_title = 'All Hands Against His Own' ALLOW FILTERING"

try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [14]:
drop_table_1 = 'DROP TABLE user_app_history'
drop_table_2 = 'DROP TABLE music_app_history'
drop_table_3 = 'DROP TABLE specific_song_listen'

try:
    session.execute(drop_table_1)
    session.execute(drop_table_2)
    session.execute(drop_table_3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()